## Procesamiento Datos Pandas

Para esta prueba deberá emplear los datos del archivo candidates.csv para lo cual deberá:

1. Almanecar los datos en una base de datos. (Asegurar el almacenamiento con el tipo de dato apropiado)
2. A partir de la lectura de los datos deberá generar un procesamiento reporte.
    
    (Cree el proceso que consideré más conveniente, realizarlo lo más complejo que le sea posible)
    
3. Realizar un código de envio de correos integrado en su solución.


El archivo candidates.csv contiene 50k filas de datos sobre candidatos. Los campos que estamos usando son:
- First Name
- Last Name
- Email
- Country
- Application Date
- Yoe (years of experience)
- Seniority
- Technology
- Code Challenge Score
- Technical Interview


In [2]:
!pip install secure-smtplib

In [23]:
!pip install matplotlib

In [66]:
import pandas as pd
import sqlite3
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication

In [50]:
archivo = './src/candidates.csv'
df = pd.read_csv(archivo, sep=';')
df.head(2)

,First Name;Last Name;Email;Application Date;Country;YOE;Seniority;Technology;Code Challenge Score;Technical Interview Score
0,Bernadette;Langworth;leonard91@yahoo.com;2021-...
1,Camryn;Reynolds;zelda56@hotmail.com;2021-09-09...


In [51]:
##df.to_excel('./output/candidatos.xlsx', index=False)

In [52]:
with sqlite3.connect("./output/candidatos.db") as conexion:
    df.to_sql("candidatos", conexion, index=False, if_exists="replace")
    
    query = 'SELECT * FROM candidatos'
    df_candidatos = pd.read_sql_query(query,conexion)
    pass

print(df_candidatos.head())

   First Name   Last Name                      Email Application Date  \
0  Bernadette   Langworth        leonard91@yahoo.com       2021-02-26   
1      Camryn    Reynolds        zelda56@hotmail.com       2021-09-09   
2       Larue      Spinka   okey_schultz41@gmail.com       2020-04-14   
3        Arch      Spinka     elvera_kulas@yahoo.com       2020-10-01   
4       Larue  Altenwerth  minnie.gislason@gmail.com       2020-05-20   

   Country  YOE  Seniority                         Technology  \
0   Norway    2     Intern                      Data Engineer   
1   Panama   10     Intern                      Data Engineer   
2  Belarus    4  Mid-Level                     Client Success   
3  Eritrea   25    Trainee                          QA Manual   
4  Myanmar   13  Mid-Level  Social Media Community Management   

   Code Challenge Score  Technical Interview Score  
0                     3                          3  
1                     2                         10  
2          

In [53]:
column_names = df_candidatos.columns
print(column_names)

Index(['First Name', 'Last Name', 'Email', 'Application Date', 'Country',
       'YOE', 'Seniority', 'Technology', 'Code Challenge Score',
       'Technical Interview Score'],
      dtype='object')


In [54]:
# Calcular estadísticas
promedio_experiencia = df_candidatos['YOE'].mean()
maxima_puntuacion = df_candidatos['Code Challenge Score'].max()
tecnologias_mas_usadas = df_candidatos['Technology'].value_counts().head(5)


In [58]:
informe = f"""
Informe de Candidatos:

- Experiencia promedio: {promedio_experiencia} años
- Puntuación máxima en el desafío de código: {maxima_puntuacion}
- Distribución de tecnología: 
{tecnologias_mas_usadas}
"""

with open("./output/informe.txt", "w") as archivo:
    archivo.write(informe)
print(informe)


Informe de Candidatos:

- Experiencia promedio: 15.28698 años
- Puntuación máxima en el desafío de código: 10
- Distribución de tecnología: 
Game Development                     3818
DevOps                               3808
Social Media Community Management    2028
System Administration                2014
Mulesoft                             1973
Name: Technology, dtype: int64



In [63]:
df.to_excel('./output/candidatos.xlsx', index=False)

In [67]:

# servidor de correo electrónico
servidor_smtp = "smtp.gmail.com"
puerto_smtp = 587
correo_emisor = "jdmendozam@unitru.edu.pe"
correo_receptor = "jefreedwmm.jm@gmail.com"
contrasena = 'irrpbvfjewfkzojq' #open("./src/contrasena.txt").read().strip() # Contraseña

# mensaje de correo
mensaje = MIMEMultipart()
mensaje["From"] = correo_emisor
mensaje["To"] = correo_receptor
mensaje["Subject"] = "Informe de Candidatos"

# informe al mensaje
with open("./output/informe.txt", "r") as archivo:
    texto_informe = archivo.read()
mensaje.attach(MIMEText(texto_informe, "plain"))

#toda la bd en un xlsx 
file_path = './output/candidatos.xlsx'
with open(file_path, "rb") as archivo_excel:
    adjunto_excel = MIMEApplication(archivo_excel.read(), _subtype="xlsx")
    adjunto_excel.add_header('content-disposition', 'attachment', filename = file_path)
    mensaje.attach(adjunto_excel)

# Envio del correo
try:
    servidor = smtplib.SMTP(servidor_smtp, puerto_smtp)
    servidor.starttls()
    servidor.login(correo_emisor, contrasena)
    servidor.sendmail(correo_emisor, correo_receptor, mensaje.as_string())
    print("El correo se envió exitosamente")
except Exception as e:
    print(f"Error al enviar el correo: {str(e)}")
finally:
    servidor.quit()


El correo se envió exitosamente
